In [4]:
pip install numpy==1.26.4


Note: you may need to restart the kernel to use updated packages.


In [7]:
import os
import cv2
import numpy as np
import mediapipe as mp

DATASET_DIR = r"dataset/TEST"   # Your dataset path
HEATMAP_DIR = "heatmaps"         # Output folder

os.makedirs(HEATMAP_DIR, exist_ok=True)

mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True)

print(" Generating improved Gaussian heatmaps...")

IMG_SIZE = 256
SIGMA = 10   # Gaussian spread


def draw_gaussian(heatmap, x, y, sigma=10):
    """Draws a Gaussian blob centered at (x, y)."""
    xx, yy = np.meshgrid(np.arange(IMG_SIZE), np.arange(IMG_SIZE))
    gaussian = np.exp(-((xx - x) ** 2 + (yy - y) ** 2) / (2 * sigma ** 2))
    heatmap += gaussian
    return heatmap


def generate_heatmap(image_path, save_path):
    img = cv2.imread(image_path)
    if img is None:
        print(" Cannot read:", image_path)
        return

    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = pose.process(img_rgb)

    # Base blank canvas
    heatmap = np.zeros((IMG_SIZE, IMG_SIZE), dtype=np.float32)

    if results.pose_landmarks:
        # Store keypoints
        points = []
        for lm in results.pose_landmarks.landmark:
            x = int(lm.x * IMG_SIZE)
            y = int(lm.y * IMG_SIZE)
            points.append((x, y))
            heatmap = draw_gaussian(heatmap, x, y, sigma=SIGMA)

        # Draw skeleton lines on heatmap
        for start, end in mp_pose.POSE_CONNECTIONS:
            x1, y1 = points[start]
            x2, y2 = points[end]
            cv2.line(heatmap, (x1, y1), (x2, y2), 1.0, 2)

    # Normalize and convert to color map
    heatmap = heatmap / heatmap.max()
    heatmap = (heatmap * 255).astype(np.uint8)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

    cv2.imwrite(save_path, heatmap)


# ----- Loop through all folders -----
for label in os.listdir(DATASET_DIR):
    class_folder = os.path.join(DATASET_DIR, label)
    if not os.path.isdir(class_folder):
        continue

    save_class_folder = os.path.join(HEATMAP_DIR, label)
    os.makedirs(save_class_folder, exist_ok=True)

    for img_name in os.listdir(class_folder):
        img_path = os.path.join(class_folder, img_name)
        save_path = os.path.join(save_class_folder, img_name)

        generate_heatmap(img_path, save_path)

print(" All improved heatmaps generated!")


NameError: name 'audio_classifier' is not defined